In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load dataset
data = pd.read_csv("titanic.csv")

# Separate features and target
X = data.drop(columns=['Survived'])
y = data['Survived']

# Define preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, ['Age', 'Fare']),
        ('cat', categorical_transformer, ['Sex', 'Embarked'])])

# Apply preprocessing to features
X_preprocessed = preprocessor.fit_transform(X)



In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Initialize and train models
models = {
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "Logistic Regression": LogisticRegression(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "XGBoost": XGBClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)


In [3]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Example of hyperparameter tuning for Random Forest using GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],.
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf_grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
rf_grid.fit(X_train, y_train)
print("Best parameters for Random Forest:", rf_gri
      d.best_params_)

# Similar hyperparameter tuning can be done for other models


Best parameters for Random Forest: {'bootstrap': True, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}


In [4]:
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline

# Create pipelines
pipelines = {
    "Random Forest": make_pipeline(preprocessor, RandomForestClassifier()),
    "Support Vector Machine": make_pipeline(preprocessor, SVC()),
    "Logistic Regression": make_pipeline(preprocessor, LogisticRegression()),
    "K-Nearest Neighbors": make_pipeline(preprocessor, KNeighborsClassifier()),
    "Decision Tree": make_pipeline(preprocessor, DecisionTreeClassifier()),
    "XGBoost": make_pipeline(preprocessor, XGBClassifier())
}

# Train ensemble model
ensemble = VotingClassifier(estimators=[(name, pipeline) for name, pipeline in pipelines.items()])
ensemble.fit(X_train, y_train)


ValueError: Specifying the columns using strings is only supported for pandas DataFrames